In [2]:
!pixi install 
# set up environment 
# to use that environment in a shell, use !pixi shell
# or, use !pixi run [my_program].py

/bin/bash: line 1: pixi: command not found


In [3]:
# or, for MAAP ADE (doesn't have pixi installed but has most deps in pangeo env) 
# !pip install paramiko

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 9.1 MB/s eta 0:00:00


# How to download archived VIIRS Active Fire data from UMD FTP server

Reference: Collection 2 Visible Infrared Imaging Radiometer Suite (VIIRS)
375-m Active Fire Product User’s Guide
Version 1.0 (May 2024) https://viirsland.gsfc.nasa.gov/PDF/VIIRS_activefire_User_Guide.pdf

From Julia: "It would be interesting to try to use the fsspec sftp file system to manage this download process."

VNP14IMG == SNPP

VJ14IMG == NOAA20

VJ24IMG == NOAA21 

VNP14IMG ML = monthly location summary files in CSV format. Naming: Vxx14IMGML.YYMM.CC.VV.csv, where xx = platform (NP, J1, J2 for SNPP, NOAA20, NOAA21), YYMM = year and month of data acquisition, CC = collection number (currently 02) and VV = product versio number (currently 02)

Date range: 20 Jan 2012 to 31 Dec 2024 if available


The VNP14IMGML product can be downloaded by connecting to an SFTP server at the University
of Maryland. The login information is as follows:

- server: fuoco.geog.umd.edu
- login: fire
- password: burnt
- directory: data/VIIRS/C2/VNP14IMGML

New VNP14IMGML files are created regularly and posted to the SFTP server above 1–2
months after all Level-2 granules for a given calendar month are generated by NASA.


In [4]:
import fsspec
import os

# connect to UMD SFTP server 
fs = fsspec.filesystem('sftp', host="fuoco.geog.umd.edu", username="fire", password="burnt")

using fsspec, we can navigate the filesystem of the remote server in typical unix fashion

`fs.ls('/')`

['/data', '/fire']

In [12]:
fs.ls('/data/VIIRS/C2/')

['/data/VIIRS/C2/VJ114IMGML',
 '/data/VIIRS/C2/VNP14IMGML',
 '/data/VIIRS/C2/VNP64A1',
 '/data/VIIRS/C2/VNP64CMQ']

In [13]:
noaa20 = fs.ls('/data/VIIRS/C2/VJ114IMGML/')
print(f"{len(noaa20)} monthly files available.\n{noaa20[0]} to {noaa20[-1]}")

87 monthly files available.
/data/VIIRS/C2/VJ114IMGML/VJ114IMGML.201804.C2.04.csv.gz to /data/VIIRS/C2/VJ114IMGML/VJ114IMGML.202506.C2.04.csv.gz


In [14]:
# we want the collection 2 data from SNPP
snpp = fs.ls('/data/VIIRS/C2/VNP14IMGML/')
print(f"{len(snpp)} monthly files availabe.\n{snpp[0]} to {snpp[-1]}")

162 monthly files availabe.
/data/VIIRS/C2/VNP14IMGML/VNP14IMGML.201201.C2.04.csv.gz to /data/VIIRS/C2/VNP14IMGML/VNP14IMGML.202506.C2.04.csv.gz


In [15]:
os.mkdir("/projects/my-public-bucket/monthly_files/VJ114IMGML/")
os.mkdir("/projects/my-public-bucket/monthly_files/VNP14IMGML/")

In [16]:
# here, we are copying the whole directory of NOAA20 monthly archive files 
# to our local machine- this may take a while 
fs.get("/data/VIIRS/C2/VJ114IMGML/", "/projects/my-public-bucket/monthly_files/VJ114IMGML/", recursive=True)
# took me 13 minutes, so not too bad 
# start: 10:28am

In [17]:
fs.get("/data/VIIRS/C2/VNP14IMGML/", "/projects/my-public-bucket/monthly_files/VNP14IMGML/", recursive=True)


In [18]:
print("done")

done


In [ ]:
# copy and then unzip

## Scratch work: 

In [ ]:
# dask dataframes can read from globs of compressed files
import dask.dataframe as dd

df = dd.read_csv("./target/*.csv.gz", compression="gzip")
df
